In [1]:
pip install keras 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

np.random.seed(121)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

Using TensorFlow backend.


In [3]:
df_train = pd.read_csv('train.csv')


In [4]:
df_train.head() 

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_train.tail() 

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41999,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
print(df_train.shape)

(42000, 785)


In [7]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
                                                random_state = 121)


In [9]:
# Feature Normalization 
X_train = X_train.astype('float32')
X_cv= X_cv.astype('float32')

X_train /= 255
X_cv /= 255


# Convert labels to One Hot Encoded
# https://keras.io/utils/
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)


In [107]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [108]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [110]:
#Behind the hidden nodes what happens is the weights are being distrubuted
Inp = Input(shape=(784,))
x1 = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x2 = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x1)
x3 = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x2)
x4 = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x3)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x4)

In [111]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total params: 297,910
Trainable params: 297,910
Non-trainable params: 0
_____________________________________________________

In [112]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

In [113]:
# We rely on the  Stochastic Gradient Descent as our optimizing methodology
# https://keras.io/losses/
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [115]:
# https://stackoverflow.com/questions/47902295/what-is-the-use-of-verbose-in-keras-while-validating-the-model
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 2s - loss: 0.1060 - accuracy: 0.9698 - val_loss: 0.1418 - val_accuracy: 0.9557
Epoch 2/20
 - 2s - loss: 0.1011 - accuracy: 0.9714 - val_loss: 0.1481 - val_accuracy: 0.9550
Epoch 3/20
 - 2s - loss: 0.0963 - accuracy: 0.9728 - val_loss: 0.1374 - val_accuracy: 0.9552
Epoch 4/20
 - 2s - loss: 0.0919 - accuracy: 0.9741 - val_loss: 0.1440 - val_accuracy: 0.9555
Epoch 5/20
 - 2s - loss: 0.0880 - accuracy: 0.9751 - val_loss: 0.1315 - val_accuracy: 0.9587
Epoch 6/20
 - 2s - loss: 0.0839 - accuracy: 0.9769 - val_loss: 0.1278 - val_accuracy: 0.9598
Epoch 7/20
 - 2s - loss: 0.0801 - accuracy: 0.9775 - val_loss: 0.1261 - val_accuracy: 0.9602
Epoch 8/20
 - 2s - loss: 0.0770 - accuracy: 0.9790 - val_loss: 0.1259 - val_accuracy: 0.9611
Epoch 9/20
 - 2s - loss: 0.0738 - accuracy: 0.9796 - val_loss: 0.1218 - val_accuracy: 0.9627
Epoch 10/20
 - 2s - loss: 0.0702 - accuracy: 0.9807 - val_loss: 0.1187 - val_accuracy: 0.9630
Epoch 11/20
 - 2s - 